##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TensorFlow Lite Model Maker가 있는 텍스트 검색기

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/lite/models/modify/model_maker/text_searcher"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/g3doc/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub에서 소스 보기</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/tensorflow/tensorflow/lite/g3doc/models/modify/model_maker/text_searcher.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드</a></td>
  <td><a href="https://tfhub.dev/google/universal-sentence-encoder-lite/2"><img src="https://www.tensorflow.org/images/hub_logo_32px.png">TF 허브 모델 보기</a></td>
</table>

이 colab 노트북에서는 [TensorFlow Lite Model Maker](https://www.tensorflow.org/lite/models/modify/model_maker) 라이브러리를 사용하여 TFLite Searcher 모델을 만드는 방법을 배울 수 있습니다. 텍스트 검색기 모델을 사용하여 앱에 대한 Sematic Search 또는 Smart Reply를 구축할 수 있습니다. 이 유형의 모델을 사용하면 텍스트 쿼리를 수행하고 웹 페이지 데이터베이스와 같은 텍스트 데이터세트에서 가장 관련성이 높은 항목을 검색할 수 있습니다. 이 모델은 URL, 페이지 제목 또는 기타 텍스트 항목 식별자와 같이 사용자가 지정하는 메타데이터를 포함하여 데이터세트에서 가장 작은 거리 점수 항목 목록을 반환합니다. 이것을 빌드한 후에는 [Task Library Searcher API](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher)를 사용하여 장치(예: Android)에 배포하여 몇 줄의 코드만으로 추론을 실행할 수 있습니다.

이 튜토리얼은 CNN/DailyMail 데이터세트를 인스턴스로 활용하여 TFLite Searcher 모델을 생성합니다. 호환되는 입력 CSV(쉼표로 구분된 값) 형식을 사용하여 고유한 데이터세트로 시도할 수 있습니다.

## 확장 가능한 Nearest Neighbor를 사용한 텍스트 검색

이 튜토리얼은 [GitHub](https://github.com/abisee/cnn-dailymail) 리포지토리에서 생성된 공개 사용이 가능한 CNN/DailyMail 비익명화 요약 데이터세트를 사용합니다. 이 데이터세트에는 300,000개 이상의 뉴스 기사가 포함되어 있어 Searcher 모델을 구축하고 텍스트 쿼리에 대한 모델 추론 중에 다양한 관련 뉴스를 반환하기에 좋습니다.

이 예의 텍스트 검색기 모델은 사전 정의된 데이터베이스에서 유사한 항목을 검색할 수 있는 [ScanNN](https://github.com/google-research/google-research/tree/master/scann)(Scalable Nearest Neighbors) 인덱스 파일을 사용합니다. ScanNN은 대규모의 효율적인 벡터 유사성 검색을 첨단 성능으로 뒷받침합니다.

이 데이터세트의 하이라이트 및 URL은 이 colab에서 모델을 생성하는 데 사용됩니다.

1. 하이라이트는 임베딩 요소 벡터를 생성하고 검색에 사용하기 위한 텍스트입니다.
2. URL은 관련 하이라이트를 검색한 후 사용자에게 표시되는 반환된 결과입니다.

이 튜토리얼에서는 이러한 데이터를 CSV 파일에 저장한 다음 CSV 파일을 사용하여 모델을 빌드합니다. 다음은 데이터세트의 몇 가지 예입니다.

하이라이트 | URL
--- | ---
Hawaiian Airlines again lands at No. 1 in on-time performance. The Airline Quality Rankings Report looks at the 14 largest U.S. airlines. ExpressJet <br> and American Airlines had the worst on-time performance. Virgin America had the best baggage  handling; Southwest had lowest complaint rate. | http://www.cnn.com/2013/04/08/travel/airline-quality-report
European football's governing body reveals list of countries bidding to host 2020 finals. The 60th anniversary edition of the finals will be hosted by 13 <br> countries. Thirty-two countries are considering bids to host 2020 matches. UEFA will announce host cities on September 25. | http://edition.cnn.com:80/2013/09/20/sport/football/football-euro-2020-bid-countries/index.html?
Once octopus-hunter Dylan Mayer has now also signed a petition of 5,000 divers banning their hunt at Seacrest Park. Decision by Washington <br> Department of Fish and Wildlife could take months. | http://www.dailymail.co.uk:80/news/article-2238423/Dylan-Mayer-Washington-considers-ban-Octopus-hunting-diver-caught-ate-Puget-Sound.html?
Galaxy was observed 420 million years after the Big Bang. found by NASA’s Hubble Space Telescope, Spitzer Space Telescope, and one of nature’s <br> own natural 'zoom lenses' in space. | http://www.dailymail.co.uk/sciencetech/article-2233883/The-furthest-object-seen-Record-breaking-image-shows-galaxy-13-3-BILLION-light-years-Earth.html


## 설정


[GitHub repo](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker)에서 Model Maker 패키지를 포함하여 필수 패키지를 설치하는 것으로 시작합니다.

In [ ]:
!sudo apt -y install libportaudio2
!pip install -q tflite-model-maker
!pip install gdown

필요한 패키지를 가져옵니다.

In [ ]:
from tflite_model_maker import searcher

### 데이터세트 준비하기

이 튜토리얼은 [GitHub 리포지토리](https://github.com/abisee/cnn-dailymail)의 CNN/Daily Mail 요약 데이터세트를 사용합니다.

먼저 cnn과 dailymail의 텍스트와 URL을 다운로드하고 압축을 풉니다. Google 드라이브에서 다운로드에 실패한 경우 몇 분 정도 기다렸다가 다시 시도하거나 수동으로 다운로드한 다음 colab에 업로드하세요.

In [ ]:
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfTHk4NFg2SndKcjQ
!gdown https://drive.google.com/uc?id=0BwmD_VLjROrfM1BxdkxVaTY2bWs

!wget -O all_train.txt https://raw.githubusercontent.com/abisee/cnn-dailymail/master/url_lists/all_train.txt
!tar xzf cnn_stories.tgz
!tar xzf dailymail_stories.tgz

그런 다음 `tflite_model_maker` 라이브러리에 로드할 수 있는 CSV 파일에 데이터를 저장합니다. 코드는 [`tensorflow_datasets`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/summarization/cnn_dailymail.py)에서 이 데이터를 로드하는 데 사용되는 논리를 기반으로 합니다. 이 colab에서 사용되는 URL이 포함되어 있지 않기 때문에 `tensorflow_dataset`을 직접 사용할 수 없습니다.

데이터를 전체 데이터세트에 대한 임베딩 요소 벡터로 처리하는 데 오랜 시간이 걸리기 때문입니다. CNN 및 Daily Mail 데이터세트의 처음 5% 스토리만 데모를 위해 기본적으로 선택됩니다. 비율을 조정하거나 CNN 및 Daily Mail 데이터세트의 50% 스토리가 포함된 사전 구축된 TFLite [모델](https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/cnn_daily_text_searcher.tflite)로 검색을 시도할 수도 있습니다.

In [ ]:
#@title Save the highlights and urls to the CSV file
#@markdown Load the highlights from the stories of CNN / Daily Mail, map urls with highlights, and save them to the CSV file.

CNN_FRACTION = 0.05 #@param {type:"number"}
DAILYMAIL_FRACTION = 0.05 #@param {type:"number"}

import csv
import hashlib
import os
import tensorflow as tf

dm_single_close_quote = u"\u2019"  # unicode
dm_double_close_quote = u"\u201d"
END_TOKENS = [
    ".", "!", "?", "...", "'", "`", '"', dm_single_close_quote,
    dm_double_close_quote, ")"
]  # acceptable ways to end a sentence


def read_file(file_path):
  """Reads lines in the file."""
  lines = []
  with tf.io.gfile.GFile(file_path, "r") as f:
    for line in f:
      lines.append(line.strip())
  return lines


def url_hash(url):
  """Gets the hash value of the url."""
  h = hashlib.sha1()
  url = url.encode("utf-8")
  h.update(url)
  return h.hexdigest()


def get_url_hashes_dict(urls_path):
  """Gets hashes dict that maps the hash value to the original url in file."""
  urls = read_file(urls_path)
  return {url_hash(url): url[url.find("id_/") + 4:] for url in urls}


def find_files(folder, url_dict):
  """Finds files corresponding to the urls in the folder."""
  all_files = tf.io.gfile.listdir(folder)
  ret_files = []
  for file in all_files:
    # Gets the file name without extension.
    filename = os.path.splitext(os.path.basename(file))[0]
    if filename in url_dict:
      ret_files.append(os.path.join(folder, file))
  return ret_files


def fix_missing_period(line):
  """Adds a period to a line that is missing a period."""
  if "@highlight" in line:
    return line
  if not line:
    return line
  if line[-1] in END_TOKENS:
    return line
  return line + "."


def get_highlights(story_file):
  """Gets highlights from a story file path."""
  lines = read_file(story_file)

  # Put periods on the ends of lines that are missing them
  # (this is a problem in the dataset because many image captions don't end in
  # periods; consequently they end up in the body of the article as run-on
  # sentences)
  lines = [fix_missing_period(line) for line in lines]

  # Separate out article and abstract sentences
  highlight_list = []
  next_is_highlight = False
  for line in lines:
    if not line:
      continue  # empty line
    elif line.startswith("@highlight"):
      next_is_highlight = True
    elif next_is_highlight:
      highlight_list.append(line)

  # Make highlights into a single string.
  highlights = "\n".join(highlight_list)

  return highlights

url_hashes_dict = get_url_hashes_dict("all_train.txt")
cnn_files = find_files("cnn/stories", url_hashes_dict)
dailymail_files = find_files("dailymail/stories", url_hashes_dict)

# The size to be selected.
cnn_size = int(CNN_FRACTION * len(cnn_files))
dailymail_size = int(DAILYMAIL_FRACTION * len(dailymail_files))
print("CNN size: %d"%cnn_size)
print("Daily Mail size: %d"%dailymail_size)

with open("cnn_dailymail.csv", "w") as csvfile:
  writer = csv.DictWriter(csvfile, fieldnames=["highlights", "urls"])
  writer.writeheader()

  for file in cnn_files[:cnn_size] + dailymail_files[:dailymail_size]:
    highlights = get_highlights(file)
    # Gets the filename which is the hash value of the url.
    filename = os.path.splitext(os.path.basename(file))[0]
    url = url_hashes_dict[filename]
    writer.writerow({"highlights": highlights, "urls": url})


## 텍스트 검색기 모델 빌드하기

데이터세트를 로드하고 데이터로 모델을 만들고 TFLite 모델을 내보내 텍스트 검색기 모델을 만듭니다.

### 1단계. 데이터세트 로드하기

Model Maker는 CSV 형식의 텍스트 데이터세트와 각 텍스트 문자열의 해당 메타데이터(예: 이 예의 URL)를 사용합니다. 사용자 지정 임베더 모델을 사용하여 텍스트 문자열을 요소 벡터에 포함합니다.

이 데모에서는 [colab](https://tfhub.dev/google/universal-sentence-encoder-lite/2)에서 이미 재학습된 첨단 문장 임베딩 모델인 [Universal Sentence Encoder](https://github.com/tensorflow/tflite-support/blob/master/tensorflow_lite_support/examples/colab/on_device_text_to_image_search_tflite.ipynb)를 사용하여 Searcher 모델을 구축합니다. 이 모델은 온디바이스 추론 성능에 최적화되어 있으며 쿼리 문자열을 포함하는 데 6ms 밖에 걸리지 않습니다(Pixel 6에서 측정). 또는 [이](https://tfhub.dev/google/lite-model/universal-sentence-encoder-qa-ondevice/1?lite-format=tflite) 양자화 버전을 사용할 수 있습니다. 이 버전은 더 작지만 각 임베딩에 38ms가 걸립니다.

In [ ]:
!wget -O universal_sentence_encoder.tflite https://storage.googleapis.com/download.tensorflow.org/models/tflite_support/searcher/text_to_image_blogpost/text_embedder.tflite

`searcher.TextDataLoader` 인스턴스를 만들고 `data_loader.load_from_csv` 메서드를 사용하여 데이터세트를 로드합니다. 이 단계는 각 텍스트에 대한 임베딩 요소 벡터를 하나씩 생성하기 때문에 ~10분 정도 걸립니다. 자신의 CSV 파일을 업로드하고 로드하여 사용자 지정 모델을 빌드할 수도 있습니다.

CSV 파일에서 텍스트 열의 이름과 메타데이터 열을 지정합니다.

- 텍스트는 임베딩 요소 벡터를 생성하는 데 사용됩니다.
- 메타데이터는 특정 텍스트를 검색할 때 표시되는 내용입니다.

다음은 위에서 생성한 CNN-DailyMail CSV 파일의 처음 4개 라인입니다.

하이라이트 | URL
--- | ---
Syrian official: Obama climbed to the top of the tree, doesn't know how to get down. Obama sends a letter to the heads of the House and Senate. Obama <br> to seek congressional approval on military action against Syria. Aim is to determine whether CW were used, not by whom, says U.N. spokesman. | http://www.cnn.com/2013/08/31/world/meast/syria-civil-war/
Usain Bolt wins third gold of world championship. Anchors Jamaica to 4x100m relay victory. Eighth gold at the championships for Bolt. Jamaica double <br> up in women's 4x100m relay. | http://edition.cnn.com/2013/08/18/sport/athletics-bolt-jamaica-gold
The employee in agency's Kansas City office is among hundreds of "virtual" workers. The employee's travel to and from the mainland U.S. last year cost <br> more than $24,000. The telecommuting program, like all GSA practices, is under review. | http://www.cnn.com:80/2012/08/23/politics/gsa-hawaii-teleworking
NEW: A Canadian doctor says she was part of a team examining Harry Burkhart in 2010. NEW: Diagnosis: "autism, severe anxiety, post-traumatic stress <br> disorder and depression" Burkhart is also suspected in a German arson probe, officials say. Prosecutors believe the German national set a string of fires <br> in Los Angeles. | http://edition.cnn.com:80/2012/01/05/justice/california-arson/index.html?


In [ ]:
data_loader = searcher.TextDataLoader.create("universal_sentence_encoder.tflite", l2_normalize=True)
data_loader.load_from_csv("cnn_dailymail.csv", text_column="highlights", metadata_column="urls")

이미지 사용 사례의 경우, `searcher.ImageDataLoader` 인스턴스를 만든 다음 `data_loader.load_from_folder`를 사용하여 폴더에서 이미지를 로드할 수 있습니다. `searcher.ImageDataLoader` 인스턴스는 쿼리를 요소 벡터로 인코딩하고 TFLite Searcher 모델과 함께 내보내는 데 활용되기 때문에 TFLite 임베더 모델에서 생성해야 합니다. 예를 들면 다음과 같습니다.

```python
data_loader = searcher.ImageDataLoader.create("mobilenet_v2_035_96_embedder_with_metadata.tflite")
data_loader.load_from_folder("food/")
```

###2단계. 검색기 모델 만들기

- ScanN 옵션을 구성합니다. 자세한 내용은 [api 문서](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker/searcher/ScaNNOptions)를 참조하세요.
- 데이터 및 ScanNN 옵션에서 검색기 모델을 생성합니다. [심층 검사](https://ai.googleblog.com/2020/07/announcing-scann-efficient-vector.html)에서 ScanN 알고리즘에 대해 자세히 알아볼 수 있습니다.

In [ ]:
scann_options = searcher.ScaNNOptions(
      distance_measure="dot_product",
      tree=searcher.Tree(num_leaves=140, num_leaves_to_search=4),
      score_ah=searcher.ScoreAH(dimensions_per_block=1, anisotropic_quantization_threshold=0.2))
model = searcher.Searcher.create_from_data(data_loader, scann_options)

위의 예에서는 다음 옵션을 정의합니다.

- `distance_measure`: "dot_product"를 사용하여 두 개의 임베딩 벡터 사이의 거리를 측정합니다. "작을수록 더 가깝다"는 개념을 유지하기 위해 실제로 **음**의 내적 값을 계산합니다.

- `tree`: 데이터세트를 140개의 파티션(대략 데이터 크기의 제곱근)으로 나누고, 그 중 4개를 검색 시 검색하는데, 이는 데이터세트의 약 3%입니다.

- `score_ah`: 공간을 절약하기 위해 float 임베딩을 동일한 차원의 int8 값으로 양자화합니다.

###3단계. TFLite 모델 내보내기

그런 다음 TFLite Searcher 모델을 내보낼 수 있습니다.

In [ ]:
model.export(
      export_filename="searcher.tflite",
      userinfo="",
      export_format=searcher.ExportFormat.TFLITE)

## 쿼리에서 TFLite 모델 테스트하기

사용자 지정 쿼리 텍스트를 사용하여 내보낸 TFLite 모델을 테스트할 수 있습니다. Searcher 모델을 사용하여 텍스트를 쿼리하려면 다음과 같이 모델을 초기화하고 텍스트 구문으로 검색을 실행합니다.

In [ ]:
from tflite_support.task import text

# Initializes a TextSearcher object.
searcher = text.TextSearcher.create_from_file("searcher.tflite")

# Searches the input query.
results = searcher.search("The Airline Quality Rankings Report looks at the 14 largest U.S. airlines.")
print(results)

다양한 플랫폼에 모델을 통합하는 방법에 대한 자세한 내용은 [작업 라이브러리 설명서](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher)를 참조하세요.

# 더 읽어보기

자세한 내용은 다음을 참조하세요.

- TensorFlow Lite Model Maker [가이드](https://www.tensorflow.org/lite/models/modify/model_maker) 및 [API 참조](https://www.tensorflow.org/lite/api_docs/python/tflite_model_maker)

- 작업 라이브러리: 배포용 [TextSearcher](https://www.tensorflow.org/lite/inference_with_metadata/task_library/text_searcher)

- 엔드 투 엔드 참조 앱: [Android](https://github.com/tensorflow/examples/tree/master/lite/examples/text_searcher/android)
